### **HuggingFace Login**

In [43]:
from huggingface_hub import login
login()

### **Import Libairies**

In [56]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import Dataset



### **Read Datasets**

In [45]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
'''

df_train = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv", index_col=0)
df_test = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv", index_col=0)


### **Short Analysis**

In [46]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


### **Preprocess Datasets**

##### _Merge columns (full)_

In [47]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
sinking,41,9
damage,41,9


In [48]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [49]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


### **Model Assessment**

In [52]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Value': [f1, precision, recall]
    }, index=['F1 Score', 'Precision', 'Recall'])
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

### **ROBERT Tokenizer**

##### _ROBERT Tokenizer + split validation_

In [ ]:
robert_tokenizer_full = RobertaTokenizer.from_pretrained("roberta-base")

robert_encodings_full = robert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
robert_labels_full = torch.tensor(list(df_train_full['target']))

robert_input_ids_train_full, robert_input_ids_valid_full, \
robert_attention_mask_train_full, robert_attention_mask_valid_full, robert_y_train_full, robert_y_valid_full = train_test_split(
    robert_encodings_full['input_ids'], 
    robert_encodings_full['attention_mask'], 
    robert_labels_full, 
    test_size=0.15, 
    stratify=robert_labels_full, 
    random_state=0
)

robert_train_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_train_full),
    'attention_mask': torch.tensor(robert_attention_mask_train_full)
}

robert_valid_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_valid_full),
    'attention_mask': torch.tensor(robert_attention_mask_valid_full)
}

robert_train_dataset_full = Dataset.from_dict({
    "input_ids": robert_train_encodings_full['input_ids'],
    "attention_mask": robert_train_encodings_full['attention_mask'],
    "labels": robert_y_train_full
})

robert_valid_dataset_full = Dataset.from_dict({
    "input_ids": robert_valid_encodings_full['input_ids'],
    "attention_mask": robert_valid_encodings_full['attention_mask'],
    "labels": robert_y_valid_full
})

robert_test_encodings_full = robert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)

robert_test_encodings_full = {
    key: torch.tensor(val) for key, val in robert_test_encodings_full.items()
}

robert_test_dataset_full = Dataset.from_dict({
    "input_ids": robert_test_encodings_full['input_ids'],
    "attention_mask": robert_test_encodings_full['attention_mask']
})

##### _Save HuggingFace ROBERT Tokenizer_

In [ ]:
robert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_tokenizer")

### **ROBERT Training from scratch**

##### _ROBERT Transformers + local save_

In [ ]:

robert_model_full = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_robert_model",
    output_dir="./robert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./robert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

robert_trainer_full = Trainer(
    model=robert_model_full,
    args=training_args,
    train_dataset=robert_train_dataset_full,
    eval_dataset=robert_valid_dataset_full
)

robert_trainer_full.train()

robert_trainer_full.save_model("robert_model") # Local

##### _Save HugggingFace trained ROBERT Transformers_ 

In [ ]:
robert_trainer_full.push_to_hub()

### **ROBERT Loading from pretrained**

In [ ]:
robert_model_full = RobertaForSequenceClassification.from_pretrained("robert_model")

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_robert_model",
    output_dir="./robert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./robert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

robert_trainer_full = Trainer(
    model=robert_model_full,
    args=training_args,
    train_dataset=robert_train_dataset_full,
    eval_dataset=robert_valid_dataset_full
)

### **ROBERT Assessment**

In [ ]:
robert_predictions_full = robert_trainer_full.predict(robert_valid_dataset_full)
robert_logits_full = robert_predictions_full.predictions
robert_y_pred_full = np.argmax(robert_logits_full, axis=1)

robert_trainer_full_assessement = evaluate_classifier(robert_y_valid_full.numpy(), robert_y_pred_full)
display(robert_trainer_full_assessement[0])
display(robert_trainer_full_assessement[1])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/transformers/training_args.py:1574: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/4854 [00:00<?, ?it/s]

{'loss': 0.7047, 'grad_norm': 4.763729572296143, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6909, 'grad_norm': 2.3180086612701416, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.6959, 'grad_norm': 4.462715148925781, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.693, 'grad_norm': 4.434909820556641, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.6917, 'grad_norm': 3.107389211654663, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6905, 'grad_norm': 5.843577861785889, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6944, 'grad_norm': 6.763678073883057, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.696, 'grad_norm': 4.3050079345703125, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6812, 'grad_norm': 10.309606552124023, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.72, 'grad_norm': 4.82548713684082, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.6682, 'grad_norm': 1.92471420

### **ROBERT Test Predictions**

##### _ROBERT Test Predictions_

In [ ]:
robert_test_predictions_full = robert_trainer_full.predict(robert_test_dataset_full)
robert_test_logits_full = robert_test_predictions_full.predictions
robert_test_y_pred_full = np.argmax(robert_test_logits_full, axis=1)

robert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': robert_test_y_pred_full.flatten()
})

##### _Save HuggingFace ROBERT Test Predictions_

In [ ]:
hf_robert_test_submission_full = Dataset.from_pandas(robert_test_submission_full)
hf_robert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_submission_df")